In [10]:
!pip install google-generativeai python-docx PyPDF2 pdf2image pytesseract

In [11]:
# First install the required libraries:
# pip install PyPDF2 pdf2image pytesseract pillow

from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract
import re

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # For text-based PDFs (using PyPDF2)
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text()
    except Exception as e:
        # For image-based PDFs (using OCR with pdf2image and pytesseract)
        # Note: Ensure Tesseract OCR is installed on your system
        # https://github.com/tesseract-ocr/tesseract#installing-tesseract
        images = convert_from_path(pdf_path)
        for img in images:
            text += pytesseract.image_to_string(img)
    return text

def preprocess_text(text):
    # Remove headers/footers using regex
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'\n\s*\n', '\n', text)  # Remove empty lines
    return text

In [12]:
paper_text = extract_text_from_pdf("/content/Addressing_the_Productivity_Paradox_in_Healthcare_with_Retrieval_Augmented_Generative_AI_Chatbots.pdf")
clean_text = preprocess_text(paper_text)

In [13]:
# Install: pip install openai transformers
from openai import OpenAI
import transformers

client = OpenAI(api_key='API-KEY')

def summarize_with_gpt4(text):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Summarize this research paper for a non-expert audience. Highlight key contributions and methods."},
            {"role": "user", "content": text[:30000]}  # Truncate to fit context window
        ]
    )
    return response.choices[0].message.content

def extract_key_concepts(text):
    # Use a pre-trained NLP model
    nlp = transformers.pipeline("ner", model="dslim/bert-base-NER")
    entities = nlp(text)
    keywords = [entity["word"] for entity in entities if entity["entity"] in ["B-ORG", "B-MISC"]]
    return list(set(keywords))

# Example usage:
summary = summarize_with_gpt4(clean_text)
keywords = extract_key_concepts(clean_text)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
def simplify_jargon(text):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Replace technical terms with simple analogies. Example: 'convolutional neural network' → 'an AI that detects patterns in images like a human eye'"},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content

def generate_analogy(concept):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Create a relatable analogy for this concept."},
            {"role": "user", "content": concept}
        ]
    )
    return response.choices[0].message.content

# Example:
simplified_text = simplify_jargon(summary)
analogy = generate_analogy("Generative Adversarial Networks")

In [8]:
print(simplified_text)

This study introduces a new method to make healthcare more efficient through the use of smart technologies. Even though technology use in healthcare has been on the rise, it hasn't always led to making work easier or faster, a predicament often coined as the "productivity puzzle".

The researchers built a smart chatbot system that can create summaries of doctor-patient discussions, provide tentative diagnoses, and even gauge how a patient is feeling emotionally. Think of it as a scribe and an assistant who takes notes during the consultation, collects clues to pinpoint potential health issues, and assesses a patient's mood - all to help make responses more personal and meaningful.

This system uses advanced language understanding techniques and pulls from a variety of different sources to provide accurate health-related assistance. It can help summarize patient discussions, assist in identifying potential illnesses, and understand a patient's emotional state, all with the aim of making

In [9]:
print(summary)

This research paper presents a new framework to improve productivity in healthcare through the application of artificial intelligence (AI). AI has been increasingly used in the healthcare sector, but despite many advances, there are still challenges, often referred to as the "productivity paradox", as increased investment in technology hasn’t always led to improved productivity.

The researchers developed a Retrieval Augmented Generative AI Chatbot framework leveraging a type of AI known as Generative AI. The framework utilizes AI tools to create consultation summaries, diagnostic insights, and emotional assessments of patients, providing more contextually appropriate responses. 

The proposed framework utilizes large-scale language models and draws from diverse external resources to provide accurate medical services. It includes features for conversational summary of patient interactions, a section for disease diagnosis, and an emotion detection module to help identify the emotional s

In [ ]:
def generate_video_script(summary: str, analogy: str, video_type) -> str:
    # Define structure based on video type
    if video_type == "reel":
        structure = """
        - Hook (15 seconds): Grab attention with a surprising fact/question
        - Problem (30 seconds): Explain the research gap
        - Analogy (35 seconds): Simplify using {}
        - Impact (30 seconds): Why this matters
        - Call-to-action (15 seconds)
        """.format(analogy)
    else:  # 5-minute YouTube video
        structure = """
        - Intro (30s): Context + thesis
        - Methodology (90s): Non-technical explanation
        - Key Findings (90s): Visualized results
        - Real-World Example (60s): {}
        - Conclusion (30s)
        """.format(analogy)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"Create a {video_type} script using this structure: {structure}"},
            {"role": "user", "content": summary}
        ]
    )
    return response.choices[0].message.content

# Example usage:
video_script = generate_video_script(simplified_text, analogy,video_type="youtube")


In [ ]:
print(video_script)

- Intro (30s): 

Hey there, friends! Today, we are diving deep into an innovative solution to a problem plaguing the healthcare industry - the productivity puzzle. So, healthcare technology, while advancing rapidly, isn't able to deliver on increased efficiency. We're exploring a promising system that works like a super-intelligent combo of trivia master and detective Sherlock Holmes to facilitate better healthcare services.

- Methodology (90s):

To understand this better, the researchers developed this system based on three major aspects. Picture yourself having a discussion with your patient. An AI buddy, the first part of the system, is there taking notes, compressing the lengthy conversation into an easy-to-understand summary. This is sort of like shorthand for medical consultations.

Next up, we have a disease detective. This part of the system utilizes an algorithm that scans health reports and, similar to observant Sherlock Holmes, spots patterns that hint at potential diseases

In [ ]:
def generate_scene_images(script: str, output_dir: str = "visuals"):
    import os
    import requests
    from PIL import Image
    from io import BytesIO

    os.makedirs(output_dir, exist_ok=True)
    scenes = script.split("\n\n")  # Split script into scenes

    image_paths = []
    for i, scene in enumerate(scenes):
        try:
            response = client.images.generate(
                model="dall-e-3",
                prompt=f"Create a colorful infographic-style image explaining: {scene}",
                size="1024x1024",
                quality="standard",
                n=1
            )
            img_url = response.data[0].url
            img_data = requests.get(img_url).content
            img_path = f"{output_dir}/scene_{i}.png"

            with open(img_path, "wb") as f:
                f.write(img_data)

            image_paths.append(img_path)
        except Exception as e:
            print(f"Error generating image for scene {i}: {e}")

    return image_paths

# Generate images for all scenes
scene_images = generate_scene_images(video_script)

KeyboardInterrupt: 

In [ ]:
def generate_narration(script: str, output_path: str = "narration.mp3"):
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",  # Most expressive voice
        input=script
    )
    response.stream_to_file(output_path)
    return output_path

# Generate audio narration
audio_file = generate_narration(video_script)


  response.stream_to_file(output_path)



In [ ]:
from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
import math

def create_video(image_paths: list, audio_path: str, output: str = "output.mp4"):
    # Calculate scene durations
    audio_clip = AudioFileClip(audio_path)
    total_duration = audio_clip.duration
    per_scene_duration = total_duration / len(image_paths)

    # Create video clips
    clips = []
    for img in image_paths:
        clip = ImageSequenceClip([img], durations=[per_scene_duration])
        clips.append(clip)

    final_video = concatenate_videoclips(clips)
    final_video = final_video.set_audio(audio_clip)

    # Format for vertical (reel) or horizontal (YouTube)
    if len(image_paths[0].split("_")) > 1 and "reel" in image_paths[0]:
        final_video = final_video.resize(height=1920).crop(x1=240, width=1080)

    final_video.write_videofile(output, fps=24)
    return output

# Assemble final video
video_output = create_video(scene_images, audio_file, "explainer_reel.mp4")

  if event.key is 'enter':



Moviepy - Building video explainer_reel.mp4.
MoviePy - Writing audio in explainer_reelTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video explainer_reel.mp4



Moviepy - Done !
Moviepy - video ready explainer_reel.mp4


In [ ]:
# --------------------------
# 5. Podcast Generation
# --------------------------
def generate_podcast_script(summary: str, analogy: str) -> str:
    """Create conversational podcast script"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"""Create a podcast dialogue:
            - Host asks 5 questions
            - Expert answers using analogy: {analogy}
            - Keep answers under 120 seconds"""},
            {"role": "user", "content": summary}
        ]
    )
    return response.choices[0].message.content

def generate_podcast_audio(script: str, output: str = "podcast.mp3") -> str:
    """Generate multi-voice podcast audio"""
    host_lines = [line.replace("Host: ", "")
                 for line in script.split("\n") if line.startswith("Host:")]
    expert_lines = [line.replace("Expert: ", "")
                   for line in script.split("\n") if line.startswith("Expert:")]

    # Generate voices
    generate_narration("\n".join(host_lines), "host_temp.mp3", voice="nova")
    generate_narration("\n".join(expert_lines), "expert_temp.mp3", voice="shimmer")

    # Mix audio
    host_audio = AudioSegment.from_file("host_temp.mp3")
    expert_audio = AudioSegment.from_file("expert_temp.mp3")
    combined = host_audio + AudioSegment.silent(500) + expert_audio
    combined.export(output, format="mp3")

    # Cleanup
    os.remove("host_temp.mp3")
    os.remove("expert_temp.mp3")

    return output
podcast_script = generate_podcast_script(summary,analogy)
podcast_file = generate_podcast_audio(podcast_script)

  response.stream_to_file(output_path)



In [ ]:
def full_pipeline(pdf_path: str, output_format: str = "video"):
    # Text extraction
    raw_text = extract_text_from_pdf(pdf_path)

    # Analysis and simplification
    summary = summarize_with_gpt4(raw_text)
    simplified = simplify_jargon(summary)
    analogy = generate_analogy(extract_key_concepts(raw_text)[0])

    # Content creation
    if output_format == "video":
        script = generate_video_script(simplified, analogy)
        images = generate_scene_images(script)
        audio = generate_narration(script)
        output = create_video(images, audio)
    elif output_format == "podcast":
        script = generate_podcast_script(simplified)
        output = generate_podcast_audio(script)

    return output

# Example usage:
final_output = full_pipeline("research_paper.pdf", output_format="video")

In [ ]:
!pip install openai PyPDF2 pdf2image pytesseract transformers moviepy pydub requests pillow

In [ ]:
def generate_podcast_script(summary: str, analogy: str) -> str:
    """Create conversational podcast script with alternating Q&A"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"""Create a podcast script with 5 question/answer pairs. Follow this exact format:

            Host: [Question 1]
            Expert: [Answer 1 using analogy: {analogy}]
            Host: [Question 2]
            Expert: [Answer 2  using analogy: {analogy}]
            Host: [Question 3]
            Expert: [Answer 3]
            Host: [Question 4]
            Expert: [Answer 4]
            Host: [Question 5]
            Expert: [Answer 5]

            Keep answers under 100 words. Use natural conversation flow."""},
            {"role": "user", "content": summary}
        ]
    )
    return response.choices[0].message.content

def generate_podcast_audio(script: str, output: str = "podcast5.mp3") -> str:
    """Generate conversational audio with alternating voices"""
    from pydub import AudioSegment

    # Split script into lines while preserving order
    lines = [line.strip() for line in script.split("\n") if line.strip()]
    combined_audio = AudioSegment.silent(duration=0)

    for line in lines:
        if line.startswith("Host:"):
            text = line.replace("Host: ", "")
            temp_file = "host_temp.mp3"
            generate_narration(text, temp_file, voice="nova")
        elif line.startswith("Expert:"):
            text = line.replace("Expert: ", "")
            temp_file = "expert_temp.mp3"
            generate_narration(text, temp_file, voice="shimmer")
        else:
            continue  # Skip malformed lines

        # Add pause between lines
        audio_segment = AudioSegment.from_file(temp_file)
        combined_audio += audio_segment + AudioSegment.silent(duration=500)
        os.remove(temp_file)  # Clean up temp file

    # Add intro/outro music
    intro_music = AudioSegment.silent(duration=1000)  # Replace with actual music file
    outro_music = AudioSegment.silent(duration=1000)
    final_audio = intro_music + combined_audio + outro_music

    final_audio.export(output, format="mp3")
    return output

# Example generated script format:
"""
Host: Why is this research important for regular people?
Expert: Think of it like teaching computers to see patterns the way humans do...
Host: How does the technology actually work?
Expert: Imagine two artists competing...
Host: What real-world applications could this have?
Expert: We're already seeing uses in...
"""

podcast_script = generate_podcast_script(summary, analogy)
print("Podcast Script:")
print(podcast_script)

podcast_file = generate_podcast_audio(podcast_script)
print(f"Podcast generated at: {podcast_file}")

Podcast Script:
Host: What exactly is Generative Adversarial Network, and what role does it play in this AI research?
Expert: Well, a Generative Adversarial Network (GAN) is kind of like a duo of a counterfeiter and a cop in a game of cat and mouse. The counterfeiter, also known as the generator, creates fake currency and tries to pass it off as real, while the cop, also known as the discriminator is working hard to discern which currency is real and which is fake. This constant cat and mouse game helps both the generator and discriminator to continually improve, much like GANs in machine learning improve to provide more accurate results.

Host: You mentioned a productivity paradox in healthcare settings as the motivation behind this project. Could you explain this further?
Expert: Of course. The productivity paradox refers to the scenario where despite investing a lot in AI and technology, the efficiency in the healthcare workspace doesn't see much of an improvement. It's like pouring

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)

  response.stream_to_file(output_path)



Podcast generated at: podcast5.mp3


In [ ]:
def generate_podcast_script(summary: str, analogy: str, key_concepts: list) -> str:
    """Generate podcast script covering full paper approach"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": f"""Create a detailed podcast script covering the ENTIRE research paper. Include:

            1. Paper's core problem/question
            2. Methodology/approach used
            3. Key technical innovations
            4. Experimental setup
            5. Main findings
            6. Real-world implications

            Format STRICTLY as:
            [Intro music]
            Host: Welcome! Today we're discussing [paper title]. Our guest is [imaginary expert name].
            Host: What problem were you trying to solve?
            Expert: [2-3 sentence answer using analogy: {analogy}]
            Host: How did you approach this?
            Expert: [Detailed methodology explanation using analogy: {analogy}]
            Host: What was the most innovative technical aspect?
            Expert: [Technical breakdown with example]
            Host: How did you validate your results?
            Expert: [Experiment description]
            Host: What surprised you most?
            Expert: [Key findings discussion]
            Host: Where could this be applied practically?
            Expert: [Real-world applications]
            [Outro music]

            Use concepts: {', '.join(key_concepts)}. Keep answers conversational."""},
            {"role": "user", "content": summary}
        ]
    )
    return response.choices[0].message.content

def generate_complete_podcast(pdf_path: str) -> str:
    """End-to-end podcast generation about full paper"""
    # Extract and process content
    raw_text = extract_text_from_pdf(pdf_path)
    summary = summarize_with_gpt4(raw_text)
    simplified = simplify_jargon(summary)
    key_concepts = extract_key_concepts(raw_text)
    analogy = generate_analogy(key_concepts[0])

    # Generate podcast components
    script = generate_podcast_script(summary, analogy, key_concepts)
    print("Generated Script:\n", script)

    # Audio production
    return generate_podcast_audio(script)

# Enhanced audio generation with pacing
def generate_podcast_audio(script: str, output: str = "full_paper_podcast.mp3") -> str:
    """Generate professional podcast audio with pacing"""
    from pydub import AudioSegment
    from pydub.effects import normalize

    # Split script into components
    segments = []
    current_speaker = None
    current_text = []

    for line in script.split("\n"):
        line = line.strip()
        if line.startswith("Host:") or line.startswith("Expert:"):
            if current_speaker:
                segments.append((current_speaker, "\n".join(current_text)))
            current_speaker = "Host" if line.startswith("Host:") else "Expert"
            current_text = [line.split(": ", 1)[1]]
        elif line and current_speaker:
            current_text.append(line)

    if current_text:
        segments.append((current_speaker, "\n".join(current_text)))

    # Generate audio with professional pacing
    combined = AudioSegment.silent(duration=1000)  # Intro silence
    for speaker, text in segments:
        temp_file = f"{speaker}_temp.mp3"
        voice = "nova" if speaker == "Host" else "shimmer"

        # Generate speech with natural pauses
        generate_narration(text, temp_file, voice=voice)
        audio = AudioSegment.from_file(temp_file)

        # Add subtle effects
        audio = normalize(audio).fade_in(200).fade_out(200)
        if speaker == "Expert":
            audio = audio.low_pass_filter(1500)  # Differentiate voices

        combined += audio + AudioSegment.silent(750)  # Natural pause
        os.remove(temp_file)

    # Add intro/outro music (replace with actual files)
    combined = AudioSegment.silent(2000) + combined + AudioSegment.silent(2000)
    combined.export(output, format="mp3")
    return output

# Usage
podcast_path = generate_complete_podcast("research_paper.pdf")
print(f"Full paper podcast generated at: {podcast_path}")